In [7]:
%pip install country_converter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 KB 1.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for country_converter: filename=country_converter-0.7.4-py3-none-any.whl size=53542 sha256=69827157275aae89519e0bfb2c85213f6305e13631f5529ca6103981abc48bdc
  Stored in directory: /Users/davidjurajszucs/Library/Caches/pip/wheels/d7/24/15/9aea58c388c3cb8c90968438c44267db6dcbac491b7b38a77b
Successfully built country_converter
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import pickle
import country_converter as coco
#import geopandas as gdp

In [68]:
def calculate_total_ru_depend(df_ru_import, df_total_import, df_import_depend):
    df_ru_import[['Freq', 'SIEC', 'Partner','Unit','Importer']] = df_ru_import['freq,siec,partner,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
    df_ru_import.drop(columns=['freq,siec,partner,unit,geo\TIME_PERIOD', 'Freq', 'Partner', 'Unit', 'SIEC'], inplace=True)

    df_ru_import = df_ru_import.set_index('Importer')
    df_ru_import = df_ru_import.replace(': ', np.nan)
    df_ru_import = df_ru_import.astype(np.float64)

    df_total_import[['Freq', 'SIEC', 'Partner','Unit','Importer']] = df_total_import['freq,siec,partner,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
    df_total_import.drop(columns=['freq,siec,partner,unit,geo\TIME_PERIOD', 'Freq', 'Partner', 'Unit', 'SIEC'], inplace=True)
    df_total_import = df_total_import.set_index('Importer')
    df_total_import = df_total_import.replace(': ', np.nan)
    df_total_import = df_total_import.astype(np.float64)

    df_import_depend[['Freq', 'SIEC','Unit','Importer']] = df_import_depend['freq,siec,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
    df_import_depend.drop(columns=['freq,siec,unit,geo\TIME_PERIOD', 'Freq', 'Unit', 'SIEC'], inplace=True)
    df_import_depend = df_import_depend.set_index('Importer')
    df_import_depend = df_import_depend.replace(': ', np.nan)
    df_import_depend = df_import_depend.astype(np.float64)
    print("df_import_dependancy: \n", df_import_depend)

    df_import_relative = df_ru_import.div(df_total_import, fill_value=0)  # This represents how much of the oil imports is from Russia
    df_import_depend_ru = df_import_depend.mul(df_import_relative, fill_value=0)  # This represents total dependency on oil also considering export and local production
    df_import_depend_ru.index = df_import_depend_ru.index.str.slice(-2)
    df_import_depend_ru.index = coco.convert(names=df_import_depend_ru.index.tolist(), to='name_short', not_found=None)
    df_import_depend_ru.index = df_import_depend_ru.index.str.replace("UK","United Kingdom")
    df_import_depend_ru.index = df_import_depend_ru.index.str.replace("EL","Greece")
    df_import_depend_ru.index.name = "Importer"

    df_import_depend_ru = df_import_depend_ru.drop(["19"])

    df_import_depend_ru = pd.melt(df_import_depend_ru.reset_index(), id_vars=['Importer'], value_vars=[(str(2000+i) + " ") for i in range(21)],var_name='Year', value_name='Import').sort_values(['Importer', 'Year'])
    df_import_depend_ru.reset_index(drop=True, inplace=True)

    return df_import_depend_ru

# 1. Imports of oil and petroleum products by partner country
## a. Dataframe: Imports of oil and petroleum products by partner country (From Russia) 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_OIL__custom_2512547/default/table?lang=en
## b. Dataframe: Imports of oil and petroleum products by partner country (From TOTAL) 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_OIL__custom_2512564/default/table?lang=en
## c. Dataframe: Relative dependency on import from Russia 2000-2020
As values of imports from Russia divided by total imports of oil and petroleum products
## d. Dataframe: Energy imports dependency - oil and petroleum products 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_ID__custom_2512645/default/table?lang=en
Calculated as (imports-exports)/gross_available_energy

In [71]:
df_oil_petrol_ru = pd.read_csv('datasets/nrg_ti_oil__custom_2512547_tabular.tsv', sep="\t" ,header = 0)
df_oil_petrol_total = pd.read_csv('datasets/nrg_ti_oil__custom_2512538_tabular.tsv', sep="\t" ,header = 0)
df_oil_petrol_depend = pd.read_csv('datasets/nrg_ind_id__custom_2512645_tabular.tsv', sep="\t" ,header = 0)
df_oil_petrol_depend_ru = calculate_total_ru_depend(df_oil_petrol_ru, df_oil_petrol_total, df_oil_petrol_depend)
with open('df_oil_petrol_ru.pickle', 'wb') as handle:
    pickle.dump(df_oil_petrol_depend_ru, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_oil_petrol_depend_ru

19 not found in ISOnumeric
EL not found in ISO2
UK not found in ISO2


df_import_dependancy: 
               2000      2001      2002      2003      2004     2005     2006   \
Importer                                                                        
AL           69.526    71.513    69.848    72.627    69.859   72.185   58.682   
AT           89.043    89.079    93.789    93.570    93.687   92.053   95.689   
BA              NaN       NaN       NaN       NaN       NaN      NaN      NaN   
BE          100.157   100.821    98.924   100.801    99.813  100.759  100.778   
BG           96.015    98.432    98.875    97.862    98.359  102.530   99.111   
CY          100.309    97.540   102.023    97.768    97.809  102.313  104.179   
CZ           95.310    97.452    93.622    95.754    93.631   97.493   96.890   
DE           94.592    96.683    94.059    97.230    95.438   97.317   95.791   
DK          -84.105   -71.347   -99.284   -99.795  -116.420 -104.447  -86.460   
EA19         97.639    96.789    96.347    97.189    96.860   97.650   97.531   
EE  

,Importer,Year,Import
0,Albania,2000,0.000000
1,Albania,2001,0.000000
2,Albania,2002,0.000000
3,Albania,2003,0.000000
4,Albania,2004,0.000000
...,...,...,...
898,United Kingdom,2016,3.221683
899,United Kingdom,2017,4.167897
900,United Kingdom,2018,3.329193
901,United Kingdom,2019,3.406691


# 2. Imports of solid fossil fuels by partner country
## a. Dataframe: Imports of solid fossil fuels by partner country (from Russia) 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_SFF__custom_2512607/default/table?lang=en
## b. Dataframe: Imports of solid fossil fuels by partner country (from TOTAL) 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_SFF__custom_2512600/default/table?lang=en
## c. Dataframe: Relative dependency on import from Russia 2000-2020
As values of imports from Russia divided by total imports of solid fossil fuels
## d. Dataframe: Energy imports dependency - solid fossil fuels 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_ID__custom_2522510/default/table?lang=en
datasets/nrg_ind_id__custom_2522510_tabular.tsv

In [62]:
df_solid_fuel_ru = pd.read_csv('datasets/nrg_ti_sff__custom_2512607_tabular.tsv', sep="\t" ,header = 0)
df_solid_fuel_total = pd.read_csv('datasets/nrg_ti_sff__custom_2512600_tabular.tsv', sep="\t" ,header = 0)
df_solid_fuel_depend = pd.read_csv('datasets/nrg_ind_id__custom_2522510_tabular.tsv', sep="\t" ,header = 0)
df_solid_fuel_depend_ru = calculate_total_ru_depend(df_solid_fuel_ru, df_solid_fuel_total, df_solid_fuel_depend)
df_solid_fuel_depend_ru
with open('df_solid_fuel_ru.pickle', 'wb') as handle:
    pickle.dump(df_solid_fuel_depend_ru, handle, protocol=pickle.HIGHEST_PROTOCOL)

19 not found in ISOnumeric
EL not found in ISO2
UK not found in ISO2


df_import_dependancy: 
              2000     2001     2002     2003     2004     2005     2006   \
Importer                                                                   
AL          58.904   75.294   75.807    9.375    9.375   16.667   16.667   
AT          83.931   89.462   91.801   86.573   98.443   99.377   94.058   
BA             NaN      NaN      NaN      NaN      NaN      NaN      NaN   
BE          91.241  106.811   86.283   97.556  101.497  101.346   95.975   
BG          35.200   35.837   38.013   35.698   40.703   36.878   35.207   
CY         102.041  111.321  124.528   96.226   68.521  121.080  116.613   
CZ         -21.976  -21.031  -17.330  -15.995  -13.844  -16.210  -16.349   
DE          25.567   29.897   29.110   29.438   32.570   31.741   37.988   
DK          94.921   96.575   88.376   98.258  101.382   94.385   93.605   
EA19        52.316   54.133   54.220   54.777   58.289   58.002   61.032   
EE         125.209   67.146   93.346  202.537   34.173   88.351 

# 3. Imports of natural gas by partner country
## a. Imports of natural gas by partner country (from Russia)
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_GAS__custom_2512614/default/table?lang=en
## b. Imports of natural gas by partner country (from TOTAL)
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_GAS__custom_2512612/default/table?lang=en
## c. Dataframe: Relative dependency on import from Russia 2000-2020

## d. Dataframe: Energy imports dependency - natural gas 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_ID__custom_2512658/default/table?lang=en

In [72]:
df_nat_gas_ru = pd.read_csv('datasets/nrg_ti_gas__custom_2512614_tabular.tsv', sep="\t" ,header = 0)
df_nat_gas_total = pd.read_csv('datasets/nrg_ti_gas__custom_2512612_tabular.tsv', sep="\t" ,header = 0)
df_nat_gas_depend = pd.read_csv('datasets/nrg_ind_id__custom_2512658_tabular.tsv', sep="\t" ,header = 0)
df_nat_gas_depend_ru = calculate_total_ru_depend(df_nat_gas_ru, df_nat_gas_total, df_nat_gas_depend)
with open('df_nat_gas_ru.pickle', 'wb') as handle:
    pickle.dump(df_nat_gas_depend_ru, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_nat_gas_depend_ru

19 not found in ISOnumeric
EL not found in ISO2
UK not found in ISO2


df_import_dependancy: 
               2000     2001      2002      2003      2004      2005   \
Importer                                                               
AL            0.000    0.000     0.000     0.000     0.000     0.000   
AT           80.582   72.177    72.239    80.219    79.901    88.453   
BA              NaN      NaN       NaN       NaN       NaN       NaN   
BE           99.321   99.653   102.055    98.897    99.925   100.517   
BG           93.542   99.739   103.995    94.247    95.835    87.653   
CY            0.000    0.000     0.000     0.000     0.000     0.000   
CZ           99.758   96.317   102.001    98.163    91.092    97.807   
DE           79.141   76.989    79.490    78.669    83.749    79.632   
DK          -64.787  -65.956   -64.496   -55.666   -79.662  -113.888   
EA19         68.828   65.048    69.545    69.865    69.678    72.274   
EE          100.000  100.000   100.000   100.000   100.000   100.000   
EL           99.062   99.215    97.416  

,Importer,Year,Import
0,Albania,2000,0.000000
1,Albania,2001,0.000000
2,Albania,2002,0.000000
3,Albania,2003,0.000000
4,Albania,2004,0.000000
...,...,...,...
898,United Kingdom,2016,0.000000
899,United Kingdom,2017,0.094446
900,United Kingdom,2018,1.601414
901,United Kingdom,2019,3.345057
